# <span style="color:#ff5f27;">🔱 TensorFlow Modeling </span>

---

## <span style="color:#ff5f27;"> 🔮 Connecting to Hopsworks Feature Store </span>

In [1]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://2176a0f0-3503-11ed-be64-b1a4781e5f0a.cloud.hopsworks.ai/p/128
Connected. Call `.close()` to terminate connection gracefully.


## <span style="color:#ff5f27;"> 🪝 Feature View and Training Dataset Retrieval </span>

In [2]:
feature_view = fs.get_feature_view(
    name = 'titanic_fv',
    version = 1
)

In [3]:
X, y = feature_view.get_training_data(1)

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X.drop(columns=["passengerid"]), y, test_size=0.33, random_state=42)

In [5]:
X_train.head()

,sex,age,pclass,sibsp,parch,fare,embarked
6,1,0.428870,-0.369365,-0.474545,-0.473674,0.050749,2
718,0,-0.108755,0.827377,-0.474545,-0.473674,0.014102,1
685,1,-0.031952,0.827377,-0.474545,-0.473674,0.018543,2
73,0,-0.108755,0.827377,-0.474545,-0.473674,0.015282,0
882,1,0.582478,-0.369365,0.432793,-0.473674,0.050749,2


In [6]:
y_train.head()

,survived
6,0
718,1
685,0
73,1
882,0


---

## <span style="color:#ff5f27;"> 🤖 Modeling</span>

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score

import warnings

# Mute warnings
warnings.filterwarnings("ignore")

In [ ]:
def build_model(lr):
  model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=[len(X_train.keys())]),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid') 
  ])

  model.compile(loss='binary_crossentropy',
                optimizer=Adam(learning_rate=lr),
                metrics=['accuracy'])

  return model

In [ ]:
np.random.seed(0)
tf.random.set_seed(0)
tf_model = build_model(0.095)

In [ ]:
history = tf_model.fit(
    X_train,
    y_train,
    verbose=2, epochs=25,batch_size=32)

In [ ]:
preds = tf_model.predict(X_test)
preds[preds > 0.5] = 1
preds[preds <= 0.5] = 0

In [ ]:
f1_score(y_test.values, preds)

## <span style="color:#ff5f27;"> 📝 Register model in Hopsworks</span>

In [ ]:
mr = project.get_model_registry()

In [ ]:
import joblib

joblib.dump(tf_model, 'model.pkl')

In [ ]:
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

input_schema = Schema(X_train)
output_schema = Schema(y_train)
model_schema = ModelSchema(input_schema=input_schema, output_schema=output_schema)

model_schema.to_dict()

In [ ]:
metrics = {
    'f1_score': f1_score(y_test.values, preds)
}

In [ ]:
model = mr.sklearn.create_model(
    name="titanic_tensorflow_model",
    metrics=metrics,
    description="TensorFlow NN.",
    input_example=X_test.sample(),
    model_schema=model_schema
)

model.save('model.pkl')

## <span style="color:#ff5f27;"> 🚀 Deployment</span>

## !! We have a problem with online-enabled Featre Groups (already reported on Jira), so we cannot proceed with Deployments.

In [ ]:
%%writefile predict_example.py
import os
import numpy as np
import hopsworks
import joblib

class Predict(object):

    def __init__(self):
        """ Initializes the serving state, reads a trained model"""        
        # get feature store handle
#         project = hopsworks.login()
#         self.fs = project.get_feature_store()
        
#         # get feature views
#         self.fv = self.fs.get_feature_view("titanic_fv", 1)
        
#         # initialise serving
#         self.fv.init_serving(1)

        # load the trained model
        self.model = joblib.load(os.environ["ARTIFACT_FILES_PATH"] + "/model.pkl")
        print("Initialization Complete")

    def predict(self, inputs):
        """ Serves a prediction request usign a trained model"""
#         feature_vector = self.fv.get_feature_vector({"passengerid": inputs[0]})
        
        
        return self.model.predict(inputs).tolist() # Numpy Arrays are not JSON serializable

In [ ]:
import os
dataset_api = project.get_dataset_api()

uploaded_file_path = dataset_api.upload("predict_example.py", "Models", overwrite=True)

predictor_script_path = os.path.join("/Projects", project.name, uploaded_file_path)

In [ ]:
model = mr.get_model("titanic_tensorflow_model", version=1)

In [ ]:
# Give it any name you want
deployment = model.deploy(
    name="titanicgradientboostingdeployment", 
    model_server="TENSORFLOW_SERVING",
    script_file=predictor_script_path,
    serving_tool = "KSERVE"
)

In [ ]:
print("Deployment: " + deployment.name)
deployment.describe()

In [ ]:
deployment.start()

In [ ]:
deployment.get_logs()

In [ ]:
data = {
    "inputs": model.input_example
}

data

In [ ]:
deployment.predict(data)

In [ ]:
tf_model.predict(model.input_example)